<a href="https://colab.research.google.com/github/pmadhyastha/INM434/blob/main/tokenisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic text processing (pre-processing)

In [ ]:
__author__ = "Pranava Madhyastha" 
__version__ = "INM434/IN3045 City, University of London, Spring 2023"

## Contents


1. [Tokenisation: a brief overview](#Tokenisation)
2. [Simple tokeniser](#Simple)
3. [Morphological Analyser](#MorphoAnalyser)
    1. [Finite state transducer (FST)](#FST)
4. [Stemming](#stemming)
    1. [Porter stemmer](#porter-stemmer)
5. [Subword tokeniser](#subwords)
    1. [Byte pair encoding](#bpe)

  


## Tokenisation: a brief overview

This is our first notebook for the module and we are going to focus on tokenisation. Please make sure to login to your respective google account to get started. 

A tokenizer is used to split an input string into separate tokens or pieces, where each token represents a meaningful element in the input string. This is one of the most important steps in NLP. This step is also called text-preprocessing. 

Tokenisation helps create a vocabulary of unique tokens overwhich we will run a variety of NLP algorithms. It mainly helps standardise the input data and help us squeeze in information from the long tail. It, in some cases, helps us to break the input text down into smaller, sometimes linguistically meaningful, units. 

In this notebook, we will first build a simple tokeniser which only splits on white space. We will then look at a toy morphological analyser. We will then build a simplified version of subword based tokeniser. 

---

How does this notebook work: 

  * There will be some cells with "TODO": you will have to fill the code for the placeholder "YOUR CODE HERE".

  * Each cell should take a few seconds to run, so if it is taking longer, there may be bug. 

# Simple tokeniser 

We will make use of the built in regular expression library in python to construct this tokeniser. Please refer to Chapter 2 (SLP: Jurafsky and Martin) references on regular expressions. 

The tokeniser below will take split input text into separate words based on word boundaries. This is defined using the regular expression pattern "\b\w+\b". The re.findall() function is used to extract all the tokens from the text that match the pattern, and the list of tokens is returned. 


In [ ]:
import re

def tokenise(text):
    # Define a regular expression pattern to split the text into tokens
    pattern = r'\b\w+\b'
    
    # Use the re.findall() function to extract all the tokens from the text
    tokens = re.findall(pattern, text)
    
    # Return the list of tokens
    return tokens

# Sample input for the tokeniser
text = "This is IN 3045/INM 434 Natural Language Procssing. This is some sample text for tokeniser"
tokens = tokenise(text)
print(tokens)

TODO: What does '\b\w+\b' mean? 

"\b" is a word boundary, "\w" is a shorthand character class that matches a "word character" (alphanumeric character or underscore), and "+" is a quantifier that matches one or more of the preceding token.


## Morphological analyser

We are now going to build a morphological analyser. A morphological analyser allows us to determine the morphological structure of a word, it outputs the inner structure of words with respect to the word's root and affixes. 

We will build the morphological analyser for "rats" => where the analyser will output the root and the operation with the affix. 

In [ ]:
# Define the FST transitions
transitions = {
    (0, 'r', 1, 'r', 'R'),
    (1, 'a', 2, 'a', 'A'),
    (2, 't', 3, 't', 'T'),
    (3, 's', 3, 's', '+Plural(S)'),
}

# Define the FST accepting states
accepting_states = [2,3]

# Define the initial state
initial_state = 0

def morphological_analyzer(word):
    # Initialize the current state and output
    current_state = initial_state
    output = ""
    
    # Iterate over each character in the word
    for char in word:
        # Check if a transition exists for the current state and character
        for start_state, in_char, end_state, out_char, out_string in transitions:
            if current_state == start_state and char == in_char:
                current_state = end_state
                output += out_string
                break
    
    # Check if the current state is an accepting state
    if current_state in accepting_states:
        return output
    else:
        return None

# Morphological analyser for rats 
word = "rats"
output = morphological_analyzer(word)
if output is not None:
    print("The morphological analysis of '{}' is '{}'".format(word, output))
else:
    print("No morphological analysis found for '{}'".format(word))

TODO: Extend the code to allow for other examples covered in the class. 
* leaves -> regular verb leave + plural operation. 
* leaf -> irregular noun + plural operation. 

What are the problems with this approach? 

## Stemming 
Stemming is a simpler type of word splitting. Their core structure is retained regardless of the actual meaning of the word. This is usually done by removing any affixes (suffix/prefix/inflections), from words. Stemming is a simple, approximate and faster way to reduce words to their root form, but it can lead to over-generalization, where words with different meanings are reduced to the same form.

In [ ]:
def simple_stemmer(word):
    # normalise the word by lowercasing
    word = word.lower()

    # we will only work over words > 2 characters.
    if len(word) <= 2:
        return word

    suffixes = ['al', 'ance', 'ence', 'able', 'ible', 'ment', 'ant', 'ent', 'ism',
                'ate', 'iti', 'ous', 'ive', 'ize']

    for suffix in suffixes:
        if word.endswith(suffix):
            word = word[:-len(suffix)] 
            break # TODO: why are we breaking here? (there are better ways of doing this)

    return word

# test this function for "capital" 
print(simple_stemmer("capital"))

TODO: Extend the list of suffixes, can you come up with other cases? 

Read porter stemmer algorithm and see what additional changes are necessary to make `simple_stemmer` better at stemming? 

## Byte pair encoding 

Sennrich, Rico, Barry Haddow, and Alexandra Birch. "Neural Machine Translation of Rare Words with Subword Units." Proceedings of the 54th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers). Vol. 1. 2016. http://www.aclweb.org/anthology/P16-1162

Main source: https://github.com/rsennrich/subword-nmt

Below is a simple version of the algorithm. 

Two stages: token learner and token segmenter. 

Let us first look at token learner, this involves: 

*  computing the frequencies of all words in a corpus (we do it synthetically here) 
*  start with characters as the basic vocab (characters seen in the corpus)
* to obtain vocabulary of n-merge operations: 
    - Obtain most frequent pairs of characters in the corpus
    - add the pair to the list of merges
    - add merged characters to the vocab 
* iterate n times

The code below performs this operation. 

In [ ]:
import re
import collections

def compute_symbol_pairs_frequency(vocab):
    #Compute the frequency of adjacent symbol pairs in the vocabulary.
    
    symbol_pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            symbol_pairs[symbols[i], symbols[i+1]] += freq
    return symbol_pairs

def merge_vocabulary(symbol_pair, vocab_in):
    #Merge the given symbol pair in the vocabulary.

    vocab_out = {}
    bigram = re.escape(' '.join(symbol_pair))
    pattern = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in vocab_in:
        word_out = pattern.sub(''.join(symbol_pair), word)
        vocab_out[word_out] = vocab_in[word]
    return vocab_out

def extract_symbol_pairs(word):
    #Return a set of symbol pairs in a word.

    symbol_pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        symbol_pairs.add((prev_char, char))
        prev_char = char
    return symbol_pairs

train_data = {'n a t u r a l </w>': 5, 'n a t u r e </w>': 2, 'l a n g u a g e </w>': 6, 'l a n g u a g e s </w>': 3}

bpe_codes = {}
bpe_codes_reverse = {}

num_merges = 10

for i in range(num_merges):
    pairs = compute_symbol_pairs_frequency(train_data)
    best_pair = max(pairs, key=pairs.get)
    train_data = merge_vocabulary(best_pair, train_data)
    
    bpe_codes[best_pair] = i
    bpe_codes_reverse[best_pair[0] + best_pair[1]] = best_pair
    
    print("new merge: {}".format(best_pair))
    print("train data: {}".format(train_data))


### Applying BPE to encode a new word

- get all character bigrams in the word. 
- find the pair of char. which appears among the char merges
- apply the merge on the word. 

In [ ]:
def bpe_encode(input_word):
    # Encode the input word using BPE (Byte Pair Encoding) algorithm.

    word_list = list(input_word) + ['</w>']
    pairs = get_bigrams(word_list)

    if not pairs:
        return input_word

    iteration = 0
    while True:
        iteration += 1
        bigram = min(pairs, key=lambda pair: bpe_codes.get(pair, float('inf')))
        if bigram not in bpe_codes:
            break
        first, second = bigram
        new_word = []
        i = 0
        while i < len(word_list):
            try:
                j = word_list.index(first, i)
                new_word.extend(word_list[i:j])
                i = j
            except:
                new_word.extend(word_list[i:])
                break

            if word_list[i] == first and i < len(word_list) - 1 and word_list[i + 1] == second:
                new_word.append(first + second)
                i += 2
            else:
                new_word.append(word_list[i])
                i += 1
        new_word = tuple(new_word)
        word_list = new_word
        if len(word_list) == 1:
            break
        else:
            pairs = get_bigrams(word_list)

    if word_list[-1] == '</w>':
        word_list = word_list[:-1]
    elif word_list[-1].endswith('</w>'):
        word_list = word_list[:-1] + (word_list[-1].replace('</w>', ''),)

    return word_list

def get_bigrams(word_list):
    # Get all bigrams in the input word list.
    return [(word_list[i], word_list[i + 1]) for i in range(len(word_list) - 1)]


# Try this: 

print(bpe_encode("naturalised"))
